# File 11: Final Error Correction Model

### Input Files:
- 09-phase-1-prediction.csv
- 10-positive-word-score.csv
- 10-negative-word-score.csv

### Output Files:


### Steps:
1. loading required libraries
1. loading required dataframes
1. making a dictionary of word-value pairs
1. calculate sentence score
1. creating a dataset
1. create X and Y to train model
1. splitting dataset for training and testing
1. training model
1. saving model

In [1]:
# loading required libraries
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from joblib import dump, load

In [4]:
# loading required dataframes
df = pd.read_csv("../db/09-phase-1-prediction.csv")
pos = pd.read_csv("../db/10-positive-word-score.csv")
neg = pd.read_csv("../db/10-negative-word-score.csv")
df.TEXT = df.TEXT.astype('str')

In [5]:
# making a dictionary of word-value pairs
lst = pos.values.tolist()
for row in neg.values.tolist() :
    lst.append(row)
    
dictionary = dict(lst)

In [6]:
# calculate sentence score
score = []

for x in range(len(df)) :
    text = df.TEXT[x]
    array = []
    for word in text.split() :
        if word in dictionary :
            array.append(dictionary[word] * 10)
        else :
            array.append(0)
    
    total = sum(array)        
    score.append(0 if total<0 else 1)

In [7]:
# creating a dataset
dataset = pd.DataFrame(
    list(zip(
        range(len(df)),
        df.USER.values.tolist(),
        df.ORIGINAL.values.tolist(),
        df.TEXT.values.tolist(),
        df.RATING.values.tolist(),
        df.OUTPUT.values.tolist(),
        df.SENTIMENT.values.tolist(),
        df.PRED1.values.tolist(),
        score)),
    columns = [ 'INDEX', 'USER', 'ORIGINAL', 'TEXT', 'RATING', 'OUTPUT', 'SENTIMENT', 'PRED1', 'SENT_SCORE'],
)
dataset.to_csv('../db/11-final-table.csv', index=False)

In [8]:
# create X and Y to train model
dataset = dataset[['RATING', 'OUTPUT', 'SENT_SCORE', 'PRED1', 'SENTIMENT']]
X = dataset[['OUTPUT', 'RATING', 'SENT_SCORE']]
Y = dataset['SENTIMENT']

In [10]:
# splitting dataset for training and testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
# training model
clf = SVC()
clf.fit(X_train, Y_train)
pred = clf.predict(X_test)

In [ ]:
# saving model
accuracy_score(Y_test, pred)*100
dump(clf1, '../classification-model/clf)